In [1]:
import re
import dill
from requests_futures.sessions import FuturesSession
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

# Scrape race pages

In [ ]:
race_ids = list(range(1, 13000))
session = FuturesSession(max_workers=8)
futures = [session.get(f'https://results.bikereg.com/race/{i}') for i in race_ids]

In [ ]:
for i, future in enumerate(futures):
    dill.dump(future.result().text, open(f'C:/data/results/{i}.pkd', 'wb'))

## Extract race metadata

In [ ]:
columns=['name', 'date', 'loc', 'json_url', 'weather', 'strava_url', 'coord']
df = pd.DataFrame(columns=columns)
for i in race_ids:
    if i % 100 == 0:
        print(i)
    if i in [12533, 12534]:  # these broke for some reason >:|
        continue
    text = dill.load(open(f'C:/data/results/{i}.pkd', 'rb'))
    data = get_data(text)
    df.loc[i] = {c: d for c, d in zip(columns, data)}

In [ ]:
df.to_pickle('C:/data/results/df.pkl')

In [3]:
df = pd.read_pickle('C:/data/results/df.pkl')

In [4]:
df.sample(5)

,name,date,loc,json_url,weather,strava_url,coord
2090,Sherman Pass Road Race,2012-07-28 00:00:00,,downloadrace.php?raceID=2091&json=1,"Clear, 85 degrees, wind 4 mph",http://app.strava.com/segments/,None
248,,,,,,,None
2329,Cafe Hollander Sussex Criterium,2012-06-03 00:00:00,,downloadrace.php?raceID=2330&json=1,"Partly Cloudy, 73 degrees, wind 13 mph",http://app.strava.com/segments/,None
12159,Herbalife Team MS Criterium Series,2016-04-05 00:00:00,"Kernersville, NC",downloadrace.php?raceID=12160&json=1,"Clear, 46 degrees, wind 5 mph",http://app.strava.com/segments/,"(36.1195182800293, -80.072509765625)"
585,Swan Island Criterium,2010-06-26 00:00:00,,downloadrace.php?raceID=586&json=1,,http://app.strava.com/segments/,None


## Download JSON results

In [22]:
urls = df['json_url']
urls.head()

2    downloadrace.php?raceID=12536&json=1
3    downloadrace.php?raceID=12537&json=1
4    downloadrace.php?raceID=12538&json=1
5    downloadrace.php?raceID=12539&json=1
6    downloadrace.php?raceID=12540&json=1
Name: json_url, dtype: object

In [31]:
url = f'https://results.bikereg.com/{urls[2]}'
df2 = pd.read_json(url)
columns = [str(i) for i in range(28)]  # Each column repeated as an "index" for some reason
df2 = df2.drop(columns=columns)

In [32]:
df2.head()

,RaceName,RaceCategoryName,Place,RacerID,FirstName,LastName,TeamName,RaceTime,Points,PriorPoints,...,TeamID,Category,Starters,ReportedAge,CalculatedAge,RacerCount,FinishPhotoUrl,License,MetaDataUrl,ResultID
0,CCNS Zwift Omnium,CAT A,1,494,Aidan,Charles,CharlesCoaching + CCAP,86,NaN,NaN,...,58644,1.0,NaN,NaN,37.0,23,NaN,,NaN,3481231
1,CCNS Zwift Omnium,CAT A,2,27730,Timothy,Ahearn,,82,NaN,NaN,...,3257,1.0,NaN,NaN,46.0,23,NaN,,NaN,3481232
2,CCNS Zwift Omnium,CAT A,3,1938,Hunter,Pronovost,CharlesCoaching + CCAP,80,NaN,NaN,...,58644,2.0,NaN,NaN,42.0,23,NaN,,NaN,3481233
3,CCNS Zwift Omnium,CAT A,4,122668,Lucas,Huesman,CT Cycling Advancement Program,80,NaN,NaN,...,36472,3.0,NaN,NaN,17.0,23,NaN,,NaN,3481234
4,CCNS Zwift Omnium,CAT A,5,74,Benjamin,Wolfe,Wolfe Racing p/b CCNS,78,NaN,NaN,...,63295,1.0,NaN,NaN,27.0,23,NaN,,NaN,3481235
